add story here

add

add

add

add


In [2]:
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim 
import folium
import requests

<h1> Importing and cleaning data </h1>

add

add

add

add

<h2> Census data </h2> 

<p> The first step in the analysis will be identifying cities where our friends could open their pizza place. We know that they'd like to relocate somewhere in California and avoid cities with less than 30k inhabitants.</p> 

<p> The best place to get a complete list of potential location is the California Census. This data is publicly available for download as an xlsx file and allows us to easily filter cities by population. </p>

<p> The main challenge will be data cleaning. An initial step has already been taken by removing all unnecessary columns directly in Excel. The next step will include removing county-level totals, empty lines, the "city" and "CPD" suffixes as well as the numerous unneeded spaces placed before and after text strings.</p>

<h2> Search interest </h2> 

<p>Knowing the population is useful but, since not everyone is interested in pizza, doesn't tell us much about the potential of each location.  The ideal solution would be to conduct surveys in each location to gauge local interest but this would be prohibitively expensive for a family business and very time-consuming. </p>

<p>An alternative solution is to use freely available data from Google Trends to compare the search levels for the keyword pizza in each location. </p>

<h4> Understanding Google Trends data </h4> 

<p>Moreover, the page uses a tree-structure that makes it impossible to view more than one geographical area at a time.  For this reason, it is possible to compare the intensity of searches between metropolitan areas of the same state, and between cities of the same metropolitan area, but it is not possible to directly compare cities located in different metropolitan areas.</p>

<p>To get around this problem it was decided to first obtain the score of each metropolitan area of California and then those for every city within each metropolitan area.  The scores of each city will then be weighted using the score of their metropolitan area to get data that is directly comparable. </p>

<p>NOTE: Google also displays the variation of data over time but in this particular case the search volume was stable over time and the average value for the past year was used</p>

<h2> Competition </h2>

<p>Another important element is understanding how much competition exists in each location. To get an idea we'll use FourSquare's API to obtain information on the number of pizza places that can be found within a 10km radius of each location. </p>
<p> </p>
<p> </p>